# Data import

data를 가져오는 부분

In [1]:
import csv
import numpy as np

with open('train.csv','r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
data_array = np.asarray(data, dtype = 'str')
print(data_array)
#print(data_array.shape)

[['PassengerId' 'Survived' 'Pclass' ... 'Fare' 'Cabin' 'Embarked']
 ['1' '0' '3' ... '7.25' '' 'S']
 ['2' '1' '1' ... '71.2833' 'C85' 'C']
 ...
 ['889' '0' '3' ... '23.45' '' 'S']
 ['890' '1' '1' ... '30' 'C148' 'C']
 ['891' '0' '3' ... '7.75' '' 'Q']]


data의 한글부분을 수치화한다.
(남자는 0, 여자는 1로)

In [2]:
quantifi_data = data_array[1:,:]
print(quantifi_data[0])

for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][4]=='male':
        quantifi_data[i][4]=float(0)
    elif quantifi_data[i][4]=='female':
        quantifi_data[i][4]=float(1)

['1' '0' '3' 'Braund, Mr. Owen Harris' 'male' '22' '1' '0' 'A/5 21171'
 '7.25' '' 'S']


test 할 데이터 또한 받아온후 동일 작업을 시행한다

In [3]:
with open('test.csv','r') as dest_f:
    data_iter1 = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data1 = [data1 for data1 in data_iter1]
data_array1 = np.asarray(data1, dtype = 'str')
print(data_array1)
#print(data_array.shape)

quantifi_data1 = data_array1[1:,:]

for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][3]=='male':
        quantifi_data1[i][3]=0
    elif quantifi_data1[i][3]=='female':
        quantifi_data1[i][3]=1

[['PassengerId' 'Pclass' 'Name' ... 'Fare' 'Cabin' 'Embarked']
 ['892' '3' 'Kelly, Mr. James' ... '7.8292' '' 'Q']
 ['893' '3' 'Wilkes, Mrs. James (Ellen Needs)' ... '7' '' 'S']
 ...
 ['1307' '3' 'Saether, Mr. Simon Sivertsen' ... '7.25' '' 'S']
 ['1308' '3' 'Ware, Mr. Frederick' ... '8.05' '' 'S']
 ['1309' '3' 'Peter, Master. Michael J' ... '22.3583' '' 'C']]


#### 성별과 동승자수, Pclass로만 분류해보자
#### x data size는 [datanum, 3(Pclass, 성별, 동승자수)]
#### y data size는 [datanum, 1]이다

In [4]:
import tensorflow as tf

x_data = np.vstack([quantifi_data[:,2], quantifi_data[:,4], quantifi_data[:,6]]).T
y_data = quantifi_data[:,1].reshape(quantifi_data.shape[0],1)
x_test_data = np.vstack([quantifi_data1[:,1], quantifi_data1[:,3], quantifi_data1[:,5]]).T
test_prediction = np.zeros([quantifi_data1.shape[0],1])

X = tf.placeholder(tf.float32, [None, 3])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([3,3]), name='weight1')
b1 = tf.Variable(tf.random_normal([3]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([3,1]), name='weight2')
b2 = tf.Variable(tf.random_normal([1]), name='bias2')
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data,Y: y_data})
        
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data,Y: y_data}), sess.run([W1, W2]))
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data,Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    #이제 실제 데이터를 집어넣어 예측해보자
    test_prediction = sess.run([hypothesis], feed_dict={X: x_test_data})
    print(test_prediction)
    

0 1.8111994 [array([[-0.6952003 , -0.76782715,  0.22903441],
       [ 0.37232202,  0.600797  , -0.18760115],
       [-1.0017165 , -0.25738436,  1.2030553 ]], dtype=float32), array([[0.342702  ],
       [0.92107147],
       [0.32955515]], dtype=float32)]
100 0.47919193 [array([[ 0.54014075, -1.8813313 ,  1.4188823 ],
       [-4.974187  , -0.34698343, -4.82792   ],
       [-3.6493614 , -1.2193102 ,  0.8464974 ]], dtype=float32), array([[-1.1697936 ],
       [ 0.22643833],
       [-2.0583265 ]], dtype=float32)]
200 0.44882506 [array([[ 0.6595379 , -1.6806747 ,  2.4679468 ],
       [-5.517159  ,  2.3516445 , -5.22732   ],
       [-3.7439988 , -0.9481375 ,  0.53637326]], dtype=float32), array([[-1.0948722],
       [ 0.9152976],
       [-3.1200404]], dtype=float32)]
300 0.4408623 [array([[ 0.06822287, -1.3789079 ,  2.7309859 ],
       [-5.593038  ,  5.5301375 , -5.12573   ],
       [-4.617404  , -0.71385723,  0.25830674]], dtype=float32), array([[-0.8363047],
       [ 1.4896106],
       [-3.

3200 0.43599325 [array([[ -2.108703  ,  -3.7420106 ,   0.20852295],
       [-10.146403  ,   5.4435654 ,  -1.0340478 ],
       [ -6.3689823 ,  -0.12577428,   0.42286706]], dtype=float32), array([[-1.542537 ],
       [ 3.2719913],
       [-5.586234 ]], dtype=float32)]
3300 0.43598452 [array([[ -2.1104562 ,  -3.6069202 ,   0.20297953],
       [-10.253061  ,   5.326487  ,  -0.99591094],
       [ -6.374208  ,  -0.13161376,   0.41341025]], dtype=float32), array([[-1.5339415],
       [ 3.307968 ],
       [-5.6555095]], dtype=float32)]
3400 0.4359827 [array([[ -2.1096666 ,  -3.534063  ,   0.19922388],
       [-10.352542  ,   5.282379  ,  -0.9895173 ],
       [ -6.403073  ,  -0.14406824,   0.41026583]], dtype=float32), array([[-1.5273273],
       [ 3.3363063],
       [-5.715572 ]], dtype=float32)]
3500 0.435969 [array([[ -2.1162505 ,  -3.3897154 ,   0.19624102],
       [-10.447911  ,   5.154139  ,  -0.94126284],
       [ -6.397585  ,  -0.15090103,   0.40203536]], dtype=float32), array([[-1.5173

6300 0.43586886 [array([[ -2.298396  ,  -2.516319  ,   0.22891869],
       [-12.4632225 ,   4.6210165 ,  -0.9516718 ],
       [ -7.330324  ,  -0.328668  ,   0.48909023]], dtype=float32), array([[-1.2839792],
       [ 4.190806 ],
       [-7.2182846]], dtype=float32)]
6400 0.4358687 [array([[ -2.3050613 ,  -2.516063  ,   0.2293218 ],
       [-12.527729  ,   4.6312165 ,  -0.9534952 ],
       [ -7.3730683 ,  -0.3302509 ,   0.49158102]], dtype=float32), array([[-1.2787728],
       [ 4.1980076],
       [-7.2655525]], dtype=float32)]
6500 0.4358904 [array([[ -2.3129113 ,  -2.5073228 ,   0.2263266 ],
       [-12.592211  ,   4.6187887 ,  -0.9929108 ],
       [ -7.435499  ,  -0.3367251 ,   0.49283412]], dtype=float32), array([[-1.2772657],
       [ 4.20388  ],
       [-7.3149443]], dtype=float32)]
6600 0.43587548 [array([[ -2.3207955 ,  -2.5058606 ,   0.23256694],
       [-12.656744  ,   4.609654  ,  -0.9534707 ],
       [ -7.409321  ,  -0.33392784,   0.49528083]], dtype=float32), array([[-1.277

9400 0.435855 [array([[ -2.5677216 ,  -2.4909966 ,   0.22061606],
       [-14.509638  ,   4.6150355 ,  -0.876997  ],
       [ -7.9905367 ,  -0.3253854 ,   0.46341562]], dtype=float32), array([[-1.2540846],
       [ 4.2205315],
       [-9.280685 ]], dtype=float32)]
9500 0.43585473 [array([[ -2.5767226 ,  -2.4915254 ,   0.223393  ],
       [-14.57696   ,   4.621624  ,  -0.8793756 ],
       [ -8.012906  ,  -0.32710114,   0.46443927]], dtype=float32), array([[-1.2521602],
       [ 4.222232 ],
       [-9.362313 ]], dtype=float32)]
9600 0.43585536 [array([[ -2.579917  ,  -2.4976006 ,   0.21052358],
       [-14.644016  ,   4.636558  ,  -0.88633156],
       [ -8.094877  ,  -0.32552555,   0.46875095]], dtype=float32), array([[-1.2436638],
       [ 4.230594 ],
       [-9.453315 ]], dtype=float32)]
9700 0.43585396 [array([[ -2.5924754 ,  -2.492209  ,   0.2175247 ],
       [-14.7111025 ,   4.6147656 ,  -0.8639592 ],
       [ -8.039649  ,  -0.3233711 ,   0.45761392]], dtype=float32), array([[-1.254

test_prediction의 값을 0과 1로 정규화 시켜준다

In [5]:
for i in range(len(test_prediction[0])):
    if test_prediction[0][i] > 0.5:
        test_prediction[0][i] = 1
    else:
        test_prediction[0][i] = 0

print(test_prediction)

[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.]

예측값들을 csv파일로 저장한다

In [6]:
f = open('prediction2.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(["PassengerId", "Survived"])
for i in range(len(test_prediction[0])):
    wr.writerow([892+i, int(test_prediction[0][i])])
    
f.close()